In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

import glob
import numpy as np
import pickle
import pandas as pd

from settings import *

In [3]:
SIGNIFICANCE_LEVEL = 5
results_dict = {}
exp_params = [(m, TS_max_dim, TS_max_len) for m in NUM_SAMPLES for TS_max_dim in DIM_STATE_SPACE for TS_max_len in LEN_SEQUENCE]
for kernel_name in KERNELS:
    for dataset_name in DATASETS:
        for m, TS_max_dim, TS_max_len in exp_params:
            result_filepath = os.path.join(RESULTS_DIR, kernel_name, '{}_M{}_L{}_D{}.pkl'.format(dataset_name, m, TS_max_len, TS_max_dim))
            if os.path.exists(result_filepath):
                with open(result_filepath, 'rb') as f:
                    result_dict = pickle.load(f)
                for hyp in ['H0', 'H1']:
                    if len(result_dict[hyp]) < NUM_REPETITIONS:
                        print('Warning: Result file {} only has {} repetitions for {}'.format(result_filepath, len(result_dict[hyp]), hyp))
                    key = (m, TS_max_len, TS_max_dim, hyp)
                    if key not in results_dict:
                        results_dict[key] = {}
                    if kernel_name not in results_dict[key]:
                        results_dict[key][kernel_name] = []
                    results = []
                    for i, rep in enumerate(result_dict[hyp]):
                        stat, hist = rep[0], rep[1]
                        perc = np.percentile(hist, 100 - SIGNIFICANCE_LEVEL)
                        results.append(int(stat > perc))
                    results_dict[key][kernel_name].append(np.mean(results))
            else:
                print('Warning: Missing result file {}...'.format(result_filepath))
                for hyp in ['H0', 'H1']:
                    key = (m, TS_max_len, TS_max_dim, hyp)
                    if key not in results_dict:
                        results_dict[key] = {}
                    if kernel_name not in results_dict[key]:
                        results_dict[key][kernel_name] = []
                    results_dict[key][kernel_name].append(float('nan'))

In [4]:
from IPython.display import display
for m, TS_max_dim, TS_max_len in exp_params:
    for hyp in ['H0', 'H1']:
        print('Samples: {}, Length: {}, Dimension: {} - {}'.format(m, TS_max_len, TS_max_dim, hyp))
        key = (m, TS_max_len, TS_max_dim, hyp)
        df = pd.DataFrame.from_dict(results_dict[key]).rename(index={i:k for i, k in enumerate(DATASETS)})
        display(df)

Samples: 30, Length: 10, Dimension: 5000 - H0


,Linear,RBF,Laplace,SignatureLinear,SignatureRBF,SignatureLaplace
EthanolConcentration,0.00,0.00,0.04,0.00,0.04,0.08
PhonemeSpectra,0.00,0.08,0.12,0.00,0.12,0.00
HandMovementDirection,0.12,0.08,0.04,0.08,0.04,0.04
SelfRegulationSCP2,0.04,0.00,0.00,0.12,0.08,0.00
SelfRegulationSCP1,0.00,0.04,0.12,0.08,0.04,0.00
Epilepsy,0.08,0.08,0.04,0.00,0.08,0.16
DuckDuckGeese,0.12,0.08,0.04,0.16,0.04,0.00
AtrialFibrillation,0.08,0.00,0.00,0.08,0.16,0.12
FaceDetection,0.00,0.00,0.04,0.12,0.04,0.08
StandWalkJump,0.08,0.00,0.08,0.04,0.04,0.04


Samples: 30, Length: 10, Dimension: 5000 - H1


,Linear,RBF,Laplace,SignatureLinear,SignatureRBF,SignatureLaplace
EthanolConcentration,0.36,0.40,0.52,0.04,0.28,0.16
PhonemeSpectra,0.20,0.20,0.12,0.08,0.24,0.28
HandMovementDirection,0.16,0.16,0.24,0.16,0.68,0.56
SelfRegulationSCP2,0.12,0.32,0.12,0.20,0.20,0.20
SelfRegulationSCP1,0.56,0.24,0.40,0.20,0.04,0.12
Epilepsy,0.76,1.00,1.00,0.88,0.96,0.88
DuckDuckGeese,1.00,1.00,1.00,1.00,1.00,1.00
AtrialFibrillation,0.68,1.00,1.00,0.92,1.00,1.00
FaceDetection,0.04,0.04,0.08,0.16,0.08,0.04
StandWalkJump,1.00,1.00,1.00,1.00,1.00,1.00


Samples: 30, Length: 100, Dimension: 5000 - H0


...

,Linear,RBF,Laplace,SignatureLinear,SignatureRBF,SignatureLaplace
EthanolConcentration,1.00,1.00,1.00,0.56,1.00,1.00
PhonemeSpectra,1.00,1.00,1.00,0.60,1.00,1.00
HandMovementDirection,1.00,1.00,1.00,1.00,1.00,1.00
SelfRegulationSCP2,0.56,0.88,0.88,0.72,1.00,1.00
SelfRegulationSCP1,1.00,1.00,1.00,1.00,1.00,1.00
Epilepsy,1.00,1.00,1.00,1.00,1.00,1.00
DuckDuckGeese,1.00,1.00,NaN,1.00,1.00,1.00
AtrialFibrillation,1.00,1.00,1.00,1.00,1.00,1.00
FaceDetection,1.00,0.84,1.00,0.00,0.08,0.12
StandWalkJump,1.00,1.00,1.00,1.00,1.00,1.00


Samples: 200, Length: 500, Dimension: 5000 - H0


,Linear,RBF,Laplace,SignatureLinear,SignatureRBF,SignatureLaplace
EthanolConcentration,0.04,0.08,0.04,0.08,0.16,0.12
PhonemeSpectra,0.08,0.04,0.04,0.12,0.08,0.04
HandMovementDirection,0.08,0.04,0.00,0.04,0.04,0.04
SelfRegulationSCP2,0.00,0.12,0.04,0.08,0.04,0.08
SelfRegulationSCP1,0.04,0.04,0.04,0.04,0.08,0.20
Epilepsy,0.12,0.08,0.08,0.12,0.08,0.00
DuckDuckGeese,0.16,0.04,NaN,0.04,0.08,0.08
AtrialFibrillation,0.04,0.04,0.08,0.00,0.08,0.04
FaceDetection,0.00,0.04,0.12,0.00,0.12,0.00
StandWalkJump,0.08,0.04,0.00,0.16,0.04,0.00


Samples: 200, Length: 500, Dimension: 5000 - H1


,Linear,RBF,Laplace,SignatureLinear,SignatureRBF,SignatureLaplace
EthanolConcentration,0.96,1.00,1.00,0.40,1.00,1.00
PhonemeSpectra,0.92,1.00,0.96,0.32,1.00,1.00
HandMovementDirection,1.00,1.00,1.00,0.96,1.00,1.00
SelfRegulationSCP2,0.36,1.00,0.96,0.68,0.92,1.00
SelfRegulationSCP1,1.00,1.00,1.00,1.00,1.00,1.00
Epilepsy,1.00,1.00,1.00,1.00,1.00,1.00
DuckDuckGeese,1.00,1.00,NaN,1.00,1.00,1.00
AtrialFibrillation,1.00,1.00,1.00,1.00,1.00,1.00
FaceDetection,0.88,0.88,0.96,0.24,0.04,0.12
StandWalkJump,1.00,1.00,1.00,1.00,1.00,1.00
